In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
import os
import cv2
import pickle
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model


from imblearn.under_sampling import RandomUnderSampler
from keras.utils.np_utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%load_ext tensorboard

In [2]:
dataPath = "../../data/OCT/OCT2017/"

In [3]:
labels_available = os.listdir(os.path.join(dataPath, "train"))
print("Total Number of Classes Detected :",len(labels_available))

labels_list = ['NORMAL',"CNV","DME","DRUSEN"]

Total Number of Classes Detected : 4


In [4]:
images=[]
y_trn=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "train", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        images.append(cv2.imread(os.path.join(xPath, y)))
        y_trn.append(labels_list.index(x))
    print(x ,end=" ")

y_train = to_categorical(y_trn,len(labels_list))

NORMAL CNV DME DRUSEN 

In [5]:
x_val=[]
y_val=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "val", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_val.append(cv2.imread(os.path.join(xPath, y)))
        y_val.append(labels_list.index(x))
    print(x ,end=" ")

y_validation = to_categorical(y_val,len(labels_list))

NORMAL CNV DME DRUSEN 

In [6]:
x_tst=[]
y_tst=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "test", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_tst.append(cv2.imread(os.path.join(xPath, y)))
        y_tst.append(labels_list.index(x))
    print(x ,end=" ")

NORMAL CNV DME DRUSEN 

In [7]:
def resizeIm(im, size):
    if im.shape[2] == 1:
        im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    return cv2.resize(im, size)

def resizeIms(x, size):
    return np.array(list(map(lambda im: resizeIm(im, size), x)))

In [8]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self,fraction, model):
        super(CustomCallback,self).__init__()
        self.fraction = fraction
        self.train_a = [];
        self.val_a =[];
        self.logPath = os.path.join(model, "log.txt")

        if not os.path.isdir(model):
            os.mkdir(model)

        with open(self.logPath,'w') as f:
            f.write('Starting of logging..\n')

        self.fig = plt.figure(figsize=(4,3))
        self.ax = plt.subplot(1,1,1)
        plt.ion()

    def on_train_begin(self,logs=None):
        self.fig.show()
        self.fig.canvas.draw()

    def on_train_end(self,logs=None):
        with open(self.logPath,'a') as f:
              f.write('End of logging..\n')
    
    def on_epoch_begin(self,epoch,logs=None):
        lr= tf.keras.backend.get_value(self.model.optimizer.lr)
        lr *= self.fraction
        tf.keras.backend.set_value(self.model.optimizer.lr,lr)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, learning rate changed to {:.4f}\n'.format(epoch,lr))
    
    def on_epoch_end(self,epoch,logs=None):
        val_acc = logs.get('val_accuracy')
        train_acc = logs.get('accuracy')
        self.train_a.append(train_acc)
        self.val_a.append(val_acc)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, training accuracy: {:.3f}, validation accuracy: {:.3f}\n'.format(epoch,train_acc,val_acc))
        self.ax.clear()
        self.ax.plot(range(1+epoch),self.train_a,label="Training")
        self.ax.plot(range(1+epoch),self.val_a,label="Validation")
        self.ax.set_xlabel('Epochs')
        self.ax.set_ylabel('Accuracy')
        self.ax.legend()
        self.fig.canvas.draw()
        self.fig.show()

## Load models

In [9]:
# xception base model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
xceptionNetModel = Xception(weights='imagenet')

# opticnet base model
opticNetModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')

# resnet base model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

resNetModel = ResNet50(weights='imagenet')

In [10]:
def emptyModelGenerator(model, newWeights=False):
    model = model.lower()
    if model == "resnet":
        x = resNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=resNetModel.input, outputs=predictions)
        size = (224, 224)
    elif model == "xception":
        x = xceptionNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=xceptionNetModel.input, outputs=predictions)
        size = (299, 299)
    elif model == "opticnet":
        newModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')
        size = (224, 224)
    if newWeights:
        newModel = tf.keras.models.clone_model(newModel)
    return newModel, size


def testPredict(model, size, name=None):
    X_test = resizeIms(x_tst, size)
    X_test = np.array(X_test)
    Y_test = np.array(y_tst)
    
    prediction = model.predict(X_test)
    preds = np.argmax(prediction, axis=1)
    acc = sum(preds == Y_test) / len(Y_test)
    print(f'Test acc for {name if name else "model"}: {acc:.6f}')
    

def computeConfussionMatrix(predictions, labels):
    num_labels = len(labels_list)
    cMatrix = np.zeros(shape=(num_labels, num_labels))
    for i in len(predictions):
        p = int(predictions[i])
        t = int(predictions[i])
        cMatrix[t, p] += 1
    print(cMatrix)
    ax = sns.heatmap(cMatrix, cmap="Blues", annot=True, fmt="d", xticklabels=labels_list, yticklabels=labels_list)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion matrix")
    plt.show();
    return cMatrix
    


In [11]:
optim = Adam(learning_rate=0.001)
epochs = 30
batch_size = 50

In [13]:
from sklearn.model_selection import train_test_split
import datetime

maxTrain = len(y_train)
#for p in [0.01, 0.025, 0.05, 0.075, 0.09]:
#for p in [0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9]:
for p in [1.0]:
    #X_trn, X_tst, y_trn, y_tst
    if p < 1:
        X_t, _, y_t, _ = train_test_split(images, y_train, test_size=1-p, random_state=123)
    else:
        X_t = images; y_t = y_train;
    print(f"Labels fraction: {sum(y_t == 1) / sum(y_train == 1)}")
    for net in ["xception", "resnet", "opticnet"]:
        print(f"Training {net} for {p}% of train size (aka {len(X_t)} images)...")
        model, size = emptyModelGenerator(net, False)
        X_trn = resizeIms(X_t, size)
        X_val = resizeIms(x_val, size)
        log_dir = f"logs/{net}/fit/{p}trainSet_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        optim = Adam(learning_rate=0.001)
        model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(X_trn, y_t, epochs=epochs, validation_data = (X_val, y_validation), batch_size=batch_size,
                    shuffle=True, max_queue_size=20,
                    use_multiprocessing=True, workers=5, 
                    callbacks=[CustomCallback(fraction=0.9, model=net), tensorboard_callback])
        model.save(f"../{net}/{net}_{epochs}epochs_{p*maxTrain} images")
        testPredict(model, size, name=net)
        del model
        del X_trn
        del X_val
        print("Done!\n" + '-'*50, end='\n\n')

Labels fraction: [1. 1. 1. 1.]
Training xception for 1.0% of train size (aka 83484 images)...


<ipython-input-8-797fd00c81c1>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
1670/1670 [==============================] - 1307s 772ms/step - loss: 0.9535 - accuracy: 0.8207 - val_loss: 0.4980 - val_accuracy: 0.9062


<ipython-input-8-797fd00c81c1>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
1670/1670 [==============================] - 1289s 772ms/step - loss: 0.3525 - accuracy: 0.9311 - val_loss: 0.1433 - val_accuracy: 1.0000
Epoch 3/30
1670/1670 [==============================] - 1295s 775ms/step - loss: 0.2152 - accuracy: 0.9512 - val_loss: 0.0696 - val_accuracy: 1.0000
Epoch 4/30
1670/1670 [==============================] - 1289s 772ms/step - loss: 0.1476 - accuracy: 0.9633 - val_loss: 0.0558 - val_accuracy: 1.0000
Epoch 5/30
1670/1670 [==============================] - 1290s 772ms/step - loss: 0.1145 - accuracy: 0.9705 - val_loss: 0.0510 - val_accuracy: 1.0000
Epoch 6/30
1670/1670 [==============================] - 1295s 775ms/step - loss: 0.0849 - accuracy: 0.9777 - val_loss: 0.0270 - val_accuracy: 1.0000
Epoch 7/30
1670/1670 [==============================] - 1301s 779ms/step - loss: 0.0680 - accuracy: 0.9823 - val_loss: 0.0321 - val_accuracy: 1.0000
Epoch 8/30
1670/1670 [==============================] - 1301s 779ms/step - loss: 0.0511 - accuracy: 0.9874

...

Epoch 2/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.4630 - accuracy: 0.8872 - val_loss: 0.1919 - val_accuracy: 1.0000
Epoch 3/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.2853 - accuracy: 0.9269 - val_loss: 0.1587 - val_accuracy: 0.9688
Epoch 4/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.2005 - accuracy: 0.9468 - val_loss: 0.1087 - val_accuracy: 0.9375
Epoch 5/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.1572 - accuracy: 0.9561 - val_loss: 0.0819 - val_accuracy: 0.9688
Epoch 6/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.1344 - accuracy: 0.9623 - val_loss: 0.0300 - val_accuracy: 1.0000
Epoch 7/30
1670/1670 [==============================] - 331s 198ms/step - loss: 0.1127 - accuracy: 0.9674 - val_loss: 0.0565 - val_accuracy: 0.9688
Epoch 8/30
1670/1670 [==============================] - 332s 199ms/step - loss: 0.0946 - accuracy: 0.9719 - val_

<ipython-input-8-797fd00c81c1>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
1670/1670 [==============================] - 2003s 1s/step - loss: 0.2919 - accuracy: 0.9144 - val_loss: 0.0393 - val_accuracy: 1.0000


<ipython-input-8-797fd00c81c1>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
1670/1670 [==============================] - 1977s 1s/step - loss: 0.1196 - accuracy: 0.9601 - val_loss: 0.0157 - val_accuracy: 1.0000
Epoch 3/30
1670/1670 [==============================] - 1976s 1s/step - loss: 0.1042 - accuracy: 0.9647 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 4/30
1670/1670 [==============================] - 1976s 1s/step - loss: 0.0864 - accuracy: 0.9707 - val_loss: 0.0268 - val_accuracy: 1.0000
Epoch 5/30
1670/1670 [==============================] - 1976s 1s/step - loss: 0.0667 - accuracy: 0.9766 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 6/30
1670/1670 [==============================] - 1976s 1s/step - loss: 0.0573 - accuracy: 0.9802 - val_loss: 0.1028 - val_accuracy: 0.9688
Epoch 7/30
1670/1670 [==============================] - 1975s 1s/step - loss: 0.0452 - accuracy: 0.9841 - val_loss: 0.0344 - val_accuracy: 0.9688
Epoch 8/30
1670/1670 [==============================] - 1976s 1s/step - loss: 0.0358 - accuracy: 0.9877 - val_loss: 0.0241 -